In [1]:
from nltk.corpus import wordnet
import pandas as pd
import random

random.seed(42)

In [2]:
file_path = 'C:/Users/SJH/OneDrive - korea.ac.kr/문서/MBTI 500.csv'
USER_SAMPLE = 1

In [3]:
data = pd.read_csv(file_path, encoding = "UTF-8")

In [4]:
if USER_SAMPLE :
    data = data.groupby('type').sample(frac = 0.005)
    data.reset_index(drop = True, inplace = True)

In [5]:
stopwords = ['estp', 'estj', 'esfp', 'esfj', 'entp', 'entj', 'enfp', 'enfj', 'istp', 'istj', 'isfp', 'isfj', 'intp', 'intj', 'infp', 'infj']

In [6]:
def new_words_list(word) :
    synonymus = set()
    for words in wordnet.synsets(word) :
        for lemma in words.lemmas() :
            x = lemma.name().replace("_","").replace("-","").lower()
            x = "".join([char for char in x if char in 'abcdefghijklmnopqrstuvwxyz '])
            if len(x) > 2 :
                synonymus.add(x)
    if word in synonymus :
        synonymus.remove(word)
    return synonymus

In [7]:
print(wordnet.synsets('conversation'))

[Synset('conversation.n.01')]


In [8]:
def replacement_words(sentence_split, num_of_words) :
    cp = sentence_split.copy()
    random.shuffle(cp)
    change_num = 0
    for wd in cp :
        if len(wd) <=2 or wd in stopwords :
            continue
        si = list(new_words_list(wd))
        if len(si) < 5 :
            continue
        change_word = random.choice(si)
        cp = [change_word if wd == wds else wds for wds in cp]
        change_num += 1
        if change_num >= num_of_words :
            break

    return " ".join(cp).split(' ')

In [9]:
tst = ['like', 'love', 'have', 'hate', 'brush', 'ear']
sen = 'like love have hate brush ear'
print(sen, replacement_words(sen.split(' '), 2))

like love have hate brush ear ['hate', 'liewith', 'havegot', 'brush', 'like', 'ear']


## main

In [10]:
data = data[data['type'].str.contains('N')]

In [11]:
data

,posts,type
0,way one way ask need bring food drink etc say ...,ENFJ
1,life experience dependant one chance mine exis...,ENFJ
2,hope nice night future keep get bright know lo...,ENFJ
3,around help emotional trouble college hour awa...,ENFJ
4,trailer game come get game footage thus rub mo...,ENFJ
...,...,...
496,water snake whatever call honestly get people ...,INTP
497,aisle le twenty minute beat hey get super pass...,INTP
498,total disuse lack understand se rather factor ...,INTP
499,tab week still least tab last week even get ne...,INTP


In [ ]:
nw_sentences = data['posts'].tolist()
nw_sentences

In [13]:
augment_nums = 9 # How many generate per sentence?

new_sentences = []
alpha = 0.05

for sens in nw_sentences :
    wds = sens.split(' ')
    # print(wds)
    for i in range(augment_nums) :
        v = int(alpha * 500)
        k = replacement_words(wds, v)
        # print(k)
        new_sentences.append(' '.join(k))

In [14]:
print(len(new_sentences), len(nw_sentences))

4374 486


## Export dataframe

In [15]:
ex = pd.DataFrame(new_sentences, columns= ['posts'])
ex['type']='N'
ex.to_csv('Aug_N.csv', index=False)